# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [12]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [13]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [14]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
#print(len(full_data_rows_list))
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [15]:
print(len(full_data_rows_list))

186


In [16]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

167


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [17]:
df=pd.read_csv('/home/workspace/event_datafile_new.csv')

In [18]:
df[df['sessionId']==139]

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
155,Des'ree,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",139,You Gotta Be,8
156,Mr Oizo,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",139,Flat 55,8
157,Tamba Trio,Kaylee,F,4,Summers,177.18812,free,"Phoenix-Mesa-Scottsdale, AZ",139,Quem Quiser Encontrar O Amor,8
158,The Mars Volta,Kaylee,F,5,Summers,380.42077,free,"Phoenix-Mesa-Scottsdale, AZ",139,Eriatarka,8
159,Infected Mushroom,Kaylee,F,6,Summers,440.26730,free,"Phoenix-Mesa-Scottsdale, AZ",139,Becoming Insane,8
160,Blue October / Imogen Heap,Kaylee,F,7,Summers,241.39710,free,"Phoenix-Mesa-Scottsdale, AZ",139,Congratulations,8
161,Girl Talk,Kaylee,F,8,Summers,160.15628,free,"Phoenix-Mesa-Scottsdale, AZ",139,Once again,8


In [19]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [20]:
# Create a Keyspace 
try:
    session.execute("""
    create keyspace if not exists udacity
    with replication={'class':'SimpleStrategy', 'replication_factor':3}
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [21]:
session.set_keyspace('udacity')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




#### Query 1

In [25]:
#Drop the table if it exists
try:
    session.execute("""
    drop table if exists song_in_session
    """)
except Exception as e:
    print(e)

In [26]:
#The data is modeled after a query to give the artist, song tile and length that was heard during each session and iteminsession 
#here primary key includes partition key (sessionId) and clustering key(itemInSession)
#Data is partitioned by sessionId and within that partition rows are ordered by itemInSession 
try :
    session.execute("""
         create table if not exists song_in_session (
                                    sessionId int, 
                                    itemInSession int,
                                    artist varchar,
                                    song varchar,
                                    length float, 
                                    primary key (sessionId, itemInSession) 
                                    )
                     """)
except Exception as e:
    print(e)                    

In [27]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        query = "insert into song_in_session (sessionId, itemInSession, artist, song, length)"
        query = query + "values (%s, %s,%s,%s,%s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [28]:
## Add in the SELECT statement to verify the data was entered into the table
rows=session.execute("select artist, song, length from song_in_session where sessionId = 338 and itemInSession = 4")
for row in rows:
    print(row.artist, row.song, row.length)

#### Query 2

In [30]:
#Drop the table if it exists
try:
    session.execute("""
    drop table if exists song_playlist_session
    """)
except Exception as e:
    print(e)

In [31]:
#The data is modeled to get what songs a particular user was listening to during a session
#here primary key includes partition key (userId) and clustering key(sessionId, itemInSession)
#Data is partitioned by userId and within that partition rows are ordered by sessionId and itemInSession

try :
    session.execute("""
         create table if not exists song_playlist_session (
                                    userId int,
                                    sessionId int, 
                                    itemInSession int,
                                    artist varchar,
                                    song varchar,
                                    firstName varchar,
                                    lastName varchar,
                                    primary key (userId, sessionId, itemInSession)
                                                         )
                     """)
except Exception as e:
    print(e)                                        

In [32]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "insert into song_playlist_session (userId, sessionId, itemInSession,artist, song, firstName, lastName)"
        query = query + "values (%s, %s,%s, %s,%s,%s,%s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1],line[4]))

In [35]:
rows=session.execute("select artist, song, firstName, lastName from song_playlist_session where userId = 10 and sessionId = 182")
for row in rows:
    print(row.artist, row.song, row.firstname, row.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


#### Query 3

In [37]:
#drop the table if it exists
try:
    session.execute("""
    drop table if exists song_played_by
    """)
except Exception as e:
    print(e)                    

In [38]:
#The data is modeled to get all users who listened to a particular song
#here primary key includes partition key (song) and clustering key(userId)
#Data is partitioned by song and within that partition rows are ordered by userId

try :
    session.execute("""
         create table if not exists song_played_by (
                                    song varchar,
                                    userId int,
                                    firstName varchar,
                                    lastName varchar,
                                    primary key (song, userId)
                                   
                                                     )
                     """)
except Exception as e:
    print(e) 

In [39]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "insert into song_played_by (song, userId, firstName, lastName)"
        query = query + "values (%s, %s,%s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1],line[4]))

In [40]:
rows=session.execute("select firstName, lastName from song_played_by where song='All Hands Against His Own'")
for row in rows:
    print(row.firstname, row.lastname)

### Drop the tables before closing out the sessions

In [32]:
session.execute("""
    drop table if exists song_in_session
    """)
session.execute("""
    drop table if exists song_playlist_session
    """)
session.execute("""
    drop table if exists song_played_by
    """)

### Close the session and cluster connection¶

In [33]:
session.shutdown()
cluster.shutdown()